In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:

PINECONE_API_KEY = "pcsk_6gMGpV_SqVtJSPFYoAdwFHSm9qXPzeZxvQ8NuF531Q3dGDmWFMc5kcJ4LokBsuVX9gpTht"
PINECONE_API_ENV = "us-east-1"

In [4]:
## Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf(r"data//")

FileNotFoundError: Directory not found: 'data//'

In [5]:
# extracted_data

In [11]:

## Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [8]:
# text_chunks

In [9]:

## download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:

embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

In [13]:
# query_result

In [ ]:
# pinecone.init(api_key=PINECONE_API_KEY,
            #   environment=PINECONE_API_ENV)

index_name = "medical-chatboat"
pc.Index(index_name)


In [33]:
from langchain.vectorstores import Pinecone


In [ ]:

docsearch = Pinecone.from_texts(
    [t.page_content for t in text_chunks],
    embedding=embeddings,
    index_name=index_name,
    namespace="",
    pinecone_api_key=PINECONE_API_KEY,
    pinecone_environment=PINECONE_API_ENV
)


In [ ]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

In [20]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [23]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])